### KRX 데이터정보시스템
매일 주식거래 내용

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import requests
import time
import os, sys

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [16]:
driver = wd.Chrome(driver_name)

In [17]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020103'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(2)

In [19]:
company_name = ['005930/삼성전자', '005380/현대차', '035420/NAVER', '033780/KT&G']
company_code = {'sec': '005930', 'hyunmotor': '005380', 'naver': '035420', 'ktng': '033780'}
company_name = ['005380/현대차']
company_code = {'hyunmotor': '005380'}

company_match = {'005930': '005930/삼성전자', '005380': '005380/현대차',
                 '035420': '035420/NAVER', '033780': '033780/KT&G'}

In [26]:
start_date = '2022-01-01'
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
end_date = today

In [ ]:
column_name = ['financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 'pension',
              'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc', 'total' ]

In [23]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
           > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
driver.find_element_by_css_selector(css_sel).click()
time.sleep(1)

In [28]:
# 회사이름 입력 Q 버튼
driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_0').click()
time.sleep(2)

# pop up된 입력창에서 회사이름 입력
driver.find_element_by_id('searchText__finder_stkisu0_0').clear()
time.sleep(1)
driver.find_element_by_id('searchText__finder_stkisu0_0').send_keys(company_name[0])
time.sleep(2)

# 검색 버튼 푸시
driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_0').click()
time.sleep(2)

# 테이블에서 최종 선택
css_sel = '#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)'
driver.find_element_by_css_selector(css_sel).click()
time.sleep(2)

# end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
driver.find_element_by_id('endDd').clear()
driver.find_element_by_id('endDd').send_keys(end_date)
time.sleep(1)
driver.find_element_by_id('strdDd').clear()
driver.find_element_by_id('strdDd').send_keys(start_date)
time.sleep(1)


In [29]:
driver.find_element_by_id('jsSearchButton').click()
time.sleep(5)

In [37]:
column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
              'vol_quantity', 'vol','total_amount', 'total_counts' ]
# ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']

In [38]:
df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
df.columns = column_name
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
df_new = df[['date', 'close', 'close_cr', 'open', 'high', 'low', 'vol']]
df_new.set_index('date', inplace=True)

In [46]:
df_new.head()

,close,close_cr,open,high,low,vol
date,,,,,,
2022-07-08,178000,1.71,177500,180500,177000,146792
2022-07-07,175000,1.45,175000,177000,172500,93811
2022-07-06,172500,-2.82,176000,177000,172000,160991
2022-07-05,177500,-0.28,180000,181000,173500,193694
2022-07-04,178000,-1.11,181000,183000,175000,179460


In [ ]:
def get_data_company(com_name, start_date, end_date):

    # 회사이름 입력 Q 버튼
    driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_0').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    driver.find_element_by_id('searchText__finder_stkisu0_0').clear()
    time.sleep(1)
    driver.find_element_by_id('searchText__finder_stkisu0_0').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
    driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_0').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)'
    driver.find_element_by_css_selector(css_sel).click()
    time.sleep(2)

    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element_by_id('endDd').clear()
    driver.find_element_by_id('endDd').send_keys(end_date)
    time.sleep(1)
    driver.find_element_by_id('strdDd').clear()
    driver.find_element_by_id('strdDd').send_keys(start_date)
    time.sleep(1)

    return get_data(date_range)

In [ ]:
column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
                  'vol_quantity', 'vol','total_amount', 'total_counts' ]
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']

def get_data(date_range):
    
    driver.find_element_by_id('jsSearchButton').click()
    time.sleep(5)
    
    df = pd.read_html(driver.page_source, 
                              attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_new = df[['date', 'close', 'close_cr', 'open', 'high', 'low', 'vol']]
    df_new.set_index('date', inplace=True)
        
    return df_new

In [ ]:
def append_data(company_code, company_match, df_new):
    for key, val in company_code.items():
        filename = 'data/{}_{}.csv'.format('data', key)
        df_company = pd.read_csv(filename)
        df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
        df_company.set_index('date', inplace=True)

    #     df_company.update(df_new, overwrite=False)
        df_company.update(df_new)
        df_company.reset_index(inplace=True)
        df_company.to_csv(filename, index=False)
    return df_company

In [ ]:
# 메인 데이터 취득
com_df = {}

for com_name in company_name:
    com_df[com_name] = get_data_company(com_name, start_date, end_date)
    
df = append_data(company_code, company_match, com_df)

In [ ]:
driver.close()
driver.quit()